In [ ]:
!pip install -q torch transformers langchain sentence-transformers tqdm openpyxl openai pandas datasets langchain-community langchain-openai langchain-core
!pip install ragatouille==0.0.9

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: ragatouille
    Found existing installation: RAGatouille 0.0.9.post2
    Uninstalling RAGatouille-0.0.9.post2:
      Successfully uninstalled RAGatouille-0.0.9.post2


In [ ]:
from tqdm.auto import tqdm
import pandas as pd
from typing import Optional, List, Tuple
import json
import datasets

pd.set_option("display.max_colwidth", None)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
ds = datasets.load_dataset("m-ric/huggingface_doc", split="train")

In [ ]:
!pip install -q langchain-text-splitters

In [ ]:
from tqdm import tqdm
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document as LangchainDocument

langchain_docs = [
    LangchainDocument(
        page_content=doc["text"],
        metadata={"source": doc["source"]}
    )
    for doc in tqdm(ds)
]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    add_start_index=True,
    separators=["\n\n", "\n", ".", " ", ""],
)

docs_processed = []
for doc in langchain_docs:
    docs_processed.extend(text_splitter.split_documents([doc]))

100%|██████████| 2647/2647 [00:00<00:00, 10513.51it/s]


In [ ]:
from huggingface_hub import InferenceClient
from google.colab import userdata

hf_token = userdata.get("HF_TOKEN")

repo_model = "mistralai/Mistral-7B-Instruct-v0.2"


llm_client = InferenceClient(
    model = repo_model,
    token = hf_token,
    timeout = 120
)

def call_llm(inference_client: InferenceClient, prompt: str):
    response = inference_client.chat.completions.create(
        messages=
          [{
              "role": "user",
              "content": prompt
          },],
        max_tokens=1000,
    )
    return response.choices[0].message.content

call_llm(llm_client, "This is a test context")

" I see. In that case, I'll assume that you want me to provide an example or explanation related to a test context in the context of software development, specifically in the context of testing frameworks like Mocha, Jest, or JUnit.\n\nA test context refers to the state and environment that is created when a test is run. It is an object that is passed as an argument to the test function, and it typically contains information about the test runner, the test file, and other related variables.\n\nFor instance, in Mocha, a test context might include the `it`, `describe`, `before`, `beforeEach`, and `after` functions, which are used to define test cases and their setup and teardown logic. It might also include a `assert` function for making assertions about the tests results.\n\nIn Jest, a test context is an object that contains information like `test`, `describe`, and `expect`, as well as `jest`, which provides access to various testing utilities.\n\nHere's an example of a test case in Moc

In [ ]:
QA_generation_prompt = """
Your task is to write a factoid question and an answer given a context.
Your factoid question should be answerable with a specific, concise piece of factual information from the context.
Your factoid question should be formulated in the same style as questions users could ask in a search engine.
This means that your factoid question MUST NOT mention something like "according to the passage" or "context".

Provide your answer as follows:

Output:::
Factoid question: (your factoid question)
Answer: (your answer to the factoid question)

Now here is the context.

Context: {context}\n
Output:::"""

In [ ]:
import random

N_GENERATIONS = 10  # We intentionally generate only 10 QA couples here for cost and time considerations

print(f"Generating {N_GENERATIONS} QA couples...")

outputs = []
for sampled_context in tqdm(random.sample(docs_processed, N_GENERATIONS)):
    # Generate QA couple
    output_QA_couple = call_llm(
        llm_client, QA_generation_prompt.format(context=sampled_context.page_content)
    )
    try:
        question = output_QA_couple.split("Factoid question: ")[-1].split("Answer: ")[0]
        answer = output_QA_couple.split("Answer: ")[-1]
        assert len(answer) < 300, "Answer is too long"
        outputs.append(
            {
                "context": sampled_context.page_content,
                "question": question,
                "answer": answer,
                "source_doc": sampled_context.metadata["source"],
            }
        )
    except:
        continue

Generating 10 QA couples...


100%|██████████| 10/10 [00:31<00:00,  3.15s/it]


In [ ]:
display(pd.DataFrame(outputs).head(1))

,context,question,answer,source_doc
0,"net = AutoModelForSequenceClassification.from_pretrained(\n CHECKPOINT, num_labels=2\n ).to(DEVICE)\n```\n\n## Federating the example\n\nThe idea behind Federated Learning is to train a model between multiple clients and a server without having to share any data. This is done by letting each client train the model locally on its data and send its parameters back to the server, which then aggregates all the clients’ parameters together using a predefined strategy. This process is made very simple by using the [Flower](https://github.com/adap/flower) framework. If you want a more complete overview, be sure to check out this guide: [What is Federated Learning?](https://flower.dev/docs/tutorial/Flower-0-What-is-FL.html)\n\n### Creating the IMDBClient\n\nTo federate our example to multiple clients, we first need to write our Flower client class (inheriting from `flwr.client.NumPyClient`). This is very easy, as our model is a standard `PyTorch` model:\n\n```python\nfrom collections import OrderedDict\n\nimport flwr as fl\n\n\nclass IMDBClient(fl.client.NumPyClient):\n def get_parameters(self, config):\n return [val.cpu().numpy() for _, val in net.state_dict().items()]\n\n def set_parameters(self, parameters):\n params_dict = zip(net.state_dict().keys(), parameters)\n state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})\n net.load_state_dict(state_dict, strict=True)\n\n def fit(self, parameters, config):\n self.set_parameters(parameters)\n print(""Training Started..."")\n train(net, trainloader, epochs=1)\n print(""Training Finished."")\n return self.get_parameters(config={}), len(trainloader), {}\n\n def evaluate(self, parameters, config):\n self.set_parameters(parameters)\n loss, accuracy = test(net, testloader)\n return float(loss), len(testloader), {""accuracy"": float(accuracy)}\n```",What is the name of the custom Flower client class created for the IMDB example?\n,IMDBClient,huggingface/blog/blob/main/fl-with-flower.md


In [ ]:
question_groundedness_critique_prompt = """
You will be given a context and a question.
Your task is to provide a 'total rating' scoring how well one can answer the given question unambiguously with the given context.
Give your answer on a scale of 1 to 5, where 1 means that the question is not answerable at all given the context, and 5 means that the question is clearly and unambiguously answerable with the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here are the question and context.

Question: {question}\n
Context: {context}\n
Answer::: """

question_relevance_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how useful this question can be to machine learning developers building NLP applications with the Hugging Face ecosystem.
Give your answer on a scale of 1 to 5, where 1 means that the question is not useful at all, and 5 means that the question is extremely useful.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

question_standalone_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how context-independent this question is.
Give your answer on a scale of 1 to 5, where 1 means that the question depends on additional information to be understood, and 5 means that the question makes sense by itself.
For instance, if the question refers to a particular setting, like 'in the context' or 'in the document', the rating must be 1.
The questions can contain obscure technical nouns or acronyms like Gradio, Hub, Hugging Face or Space and still be a 5: it must simply be clear to an operator with access to documentation what the question is about.

For instance, "What is the name of the checkpoint from which the ViT model is imported?" should receive a 1, since there is an implicit mention of a context, thus the question is not independent from the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

In [ ]:
import re

print("Generating critique for each QA couple...")

for output in tqdm(outputs):
    critique_inputs = {
        "groundedness": question_groundedness_critique_prompt.format(
            context=output["context"], question=output["question"]
        ),
        "relevance": question_relevance_critique_prompt.format(question=output["question"]),
        "standalone": question_standalone_critique_prompt.format(question=output["question"]),
    }

    for criterion, prompt in critique_inputs.items():
        try:
            evaluation = call_llm(llm_client, prompt)

            score_match = re.search(r"Total rating:\s*(\d+)", evaluation, re.IGNORECASE)

            eval_match = re.search(r"Evaluation:\s*(.+?)(?=\nTotal rating:|$)", evaluation, re.IGNORECASE | re.DOTALL)

            if score_match:
                score = int(score_match.group(1))
            else:
                score = None

            if eval_match:
                explanation = eval_match.group(1).strip()
            else:
                explanation = evaluation

            output.update({
                f"{criterion}_score": score,
                f"{criterion}_eval": explanation,
            })

        except Exception as e:
            print(f"Error critiquing {criterion}: {e}")
            output.update({
                f"{criterion}_score": None,
                f"{criterion}_eval": "Error",
            })
if outputs:
    print("Keys in first item:", outputs[0].keys())

Generating critique for each QA couple...


100%|██████████| 10/10 [02:42<00:00, 16.21s/it]

Keys in first item: dict_keys(['context', 'question', 'answer', 'source_doc', 'groundedness_score', 'groundedness_eval', 'relevance_score', 'relevance_eval', 'standalone_score', 'standalone_eval'])


In [ ]:
import pandas as pd

pd.set_option("display.max_colwidth", None)

generated_questions = pd.DataFrame.from_dict(outputs)

print("Evaluation dataset before filtering:")
display(
    generated_questions[
        [
            "question",
            "answer",
            "groundedness_score",
            "relevance_score",
            "standalone_score",
        ]
    ]
)
generated_questions = generated_questions.loc[
    (generated_questions["groundedness_score"] >= 4)
    & (generated_questions["relevance_score"] >= 4)
    & (generated_questions["standalone_score"] >= 4)
]
print("============================================")
print("Final evaluation dataset:")
display(
    generated_questions[
        [
            "question",
            "answer",
            "groundedness_score",
            "relevance_score",
            "standalone_score",
        ]
    ]
)

eval_dataset = datasets.Dataset.from_pandas(
    generated_questions, split="train", preserve_index=False
)

Evaluation dataset before filtering:


,question,answer,groundedness_score,relevance_score,standalone_score
0,What is the name of the custom Flower client class created for the IMDB example?\n,IMDBClient,1,3,1
1,What is the recommended value for the `--snr_gamma` parameter when using the Min-SNR weighting strategy in PyTorch for training with the Hugging Face script?\n,The recommended value for the `--snr_gamma` parameter when using the Min-SNR weighting strategy in PyTorch for training with the Hugging Face script is 5.0.,1,4,3
2,What metric does poseval use to compute scores for POS tagging evaluation?\n,"poseval uses scikit-learn's classification report to compute precision, recall and F1-score for POS tagging evaluation.",5,5,5
3,Which two ways does FlashAttention-2 improve the standard attention mechanism for faster inference?\n,FlashAttention-2 parallelizes the attention computation over sequence length and partitions the work between GPU threads to reduce communication and shared memory reads/writes.,5,5,1
4,What's new in version 3.19.0 of the gradio library?\n,"The new features include an improved embedding experience with configurable UI and error handling, as well as a new `gr.BarPlot` component.",5,4,3
5,In which GitHub pull requests were improvements made to the behavior of `gr.load()` with `gr.Examples` and setting `share=True` on Spaces or in wasm?\n,Pull requests #5690 and #5696 were the ones where the improvements were made.,1,4,1
6,What is the final shape of the tensor after applying the `PerceiverImagePreprocessor` in the given context?\n,"The final shape of the tensor after applying the `PerceiverImagePreprocessor` is (batch\_size, 50176, 512).",5,4,1
7,Which two tasks are shown in the context for fine-tuning models using ONNX Runtime?\n,Text classification and summarization,5,5,5
8,What mathematical expression is used to compute the mean of the DATA given a time step t and neural network parameters θ?\n,"The mean is computed as μθ(xt, t) = (1/(√αt) \* (xt - βt/\*√(1 - Åt) \* εθ(xt, t)) where αt := 1 - βt, Åt := PiSTαs, and εθ is the output of the noise predictor neural network.",5,5,5
9,What is the top 1 accuracy of the legacy\_senet154 model on ImageNet dataset?\n,The top 1 accuracy of the legacy\_senet154 model on ImageNet dataset is 81.33%.,5,1,1


Final evaluation dataset:


,question,answer,groundedness_score,relevance_score,standalone_score
2,What metric does poseval use to compute scores for POS tagging evaluation?\n,"poseval uses scikit-learn's classification report to compute precision, recall and F1-score for POS tagging evaluation.",5,5,5
7,Which two tasks are shown in the context for fine-tuning models using ONNX Runtime?\n,Text classification and summarization,5,5,5
8,What mathematical expression is used to compute the mean of the DATA given a time step t and neural network parameters θ?\n,"The mean is computed as μθ(xt, t) = (1/(√αt) \* (xt - βt/\*√(1 - Åt) \* εθ(xt, t)) where αt := 1 - βt, Åt := PiSTαs, and εθ is the output of the noise predictor neural network.",5,5,5


In [ ]:
eval_dataset = datasets.load_dataset("m-ric/huggingface_doc_qa_eval", split="train")

2.1. Preprocessing documents to build our vector database
In this part, we split the documents from our knowledge base into smaller chunks: these will be the snippets that are picked by the Retriever, to then be ingested by the Reader LLM as supporting elements for its answer.
The goal is to build semantically relevant snippets: not too small to be sufficient for supporting an answer, and not too large too avoid diluting individual ideas.
Many options exist for text splitting:

split every n words / characters, but this has the risk of cutting in half paragraphs or even sentences
split after n words / character, but only on sentence boundaries
recursive split tries to preserve even more of the document structure, by processing it tree-like way, splitting first on the largest units (chapters) then recursively splitting on smaller units (paragraphs, sentences).
To learn more about chunking, I recommend you read this great notebook by Greg Kamradt.

This space lets you visualize how different splitting options affect the chunks you get.

In the following, we use Langchain's RecursiveCharacterTextSplitter.

💡 To measure chunk length in our Text Splitter, our length function will not be the count of characters, but the count of tokens in the tokenized text: indeed, for subsequent embedder that processes token, measuring length in tokens is more relevant and empirically performs better.

In [ ]:
from langchain_core.documents import Document as LangchainDocument

RAW_KNOWLEDGE_BASE = [
    LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]})
    for doc in tqdm(ds)
]

100%|██████████| 2647/2647 [00:00<00:00, 17719.77it/s]


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer


def split_documents(
    chunk_size: int,
    knowledge_base: List[LangchainDocument],
    tokenizer_name: str,
) -> List[LangchainDocument]:
    """
    Split documents into chunks of size `chunk_size` characters and return a list of documents.
    """
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(tokenizer_name),
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size / 10),
        add_start_index=True,
        strip_whitespace=True,
        separators=["\n\n", "\n", ".", " ", ""],
    )

    docs_processed = []
    for doc in knowledge_base:
        docs_processed += text_splitter.split_documents([doc])

    # Remove duplicates
    unique_texts = {}
    docs_processed_unique = []
    for doc in docs_processed:
        if doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            docs_processed_unique.append(doc)

    return docs_processed_unique

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
import os

def load_embeddings(
    langchain_docs: List[LangchainDocument],
    chunk_size: int,
    embedding_model_name: Optional[str] = "thenlper/gte-small",
) -> FAISS:
    # load embedding_model
    embedding_model = HuggingFaceEmbeddings(
        model_name=embedding_model_name,
        multi_process=True,
        model_kwargs={"device": "cuda"},
        encode_kwargs={
            "normalize_embeddings": True
        },
    )

    # Check if embeddings already exist on disk
    index_name = (
        f"index_chunk:{chunk_size}_embeddings:{embedding_model_name.replace('/', '~')}"
    )
    index_folder_path = f"./data/indexes/{index_name}/"
    if os.path.isdir(index_folder_path):
        return FAISS.load_local(
            index_folder_path,
            embedding_model,
            distance_strategy=DistanceStrategy.COSINE,
            allow_dangerous_deserialization=True
        )

    else:
        print("Index not found, generating it...")
        docs_processed = split_documents(
            chunk_size,
            langchain_docs,
            embedding_model_name,
        )
        knowledge_index = FAISS.from_documents(
            docs_processed, embedding_model, distance_strategy=DistanceStrategy.COSINE
        )
        knowledge_index.save_local(index_folder_path)
        return knowledge_index

In [ ]:
RAG_PROMPT_TEMPLATE = """
<|system|>
Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.</s>
<|user|>
Context:
{context}
---
Now here is the question you need to answer.

Question: {question}
</s>
<|assistant|>
"""

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

repo_id = "HuggingFaceH4/zephyr-7b-beta"

base_llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    task="text-generation",
    huggingfacehub_api_token=hf_token,
    max_new_tokens=512,
    temperature=0.1,
    repetition_penalty=1.03,
)

READER_LLM = ChatHuggingFace(llm=base_llm)

In [ ]:
from ragatouille import RAGPretrainedModel
from langchain_core.vectorstores import VectorStore
from langchain_core.language_models.llms import LLM

def answer_with_rag(
    question: str,
    llm: LLM,
    knowledge_index: VectorStore,
    reranker: Optional[RAGPretrainedModel] = None,
    num_retrieved_docs: int = 30,
    num_docs_final: int = 7,
) -> Tuple[str, List[LangchainDocument]]:
    """Answer a question using RAG with the given knowledge index."""
    # Gather documents with retriever
    relevant_docs = knowledge_index.similarity_search(
        query=question, k=num_retrieved_docs
    )
    relevant_docs = [doc.page_content for doc in relevant_docs]  # keep only the text

    # Optionally rerank results
    if reranker:
        relevant_docs = reranker.rerank(question, relevant_docs, k=num_docs_final)
        relevant_docs = [doc["content"] for doc in relevant_docs]

    relevant_docs = relevant_docs[:num_docs_final]

    # Build the final prompt
    context = "\nExtracted documents:\n"
    context += "".join(
        [f"Document {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs)]
    )

    final_prompt = RAG_PROMPT_TEMPLATE.format(question=question, context=context)

    answer = llm.invoke(final_prompt)

    return answer, relevant_docs

ModuleNotFoundError: No module named 'langchain.retrievers'

In [ ]:
from langchain_core.language_models import BaseChatModel

def run_rag_tests(
    eval_dataset: datasets.Dataset,
    llm,
    knowledge_index: VectorStore,
    output_file: str,
    reranker: Optional[RAGPretrainedModel] = None,
    verbose: Optional[bool] = True,
    test_settings: Optional[str] = None,  # To document the test settings used
):
    """Runs RAG tests on the given dataset and saves the results to the given output file."""
    try:  # load previous generations if they exist
        with open(output_file, "r") as f:
            outputs = json.load(f)
    except:
        outputs = []

    for example in tqdm(eval_dataset):
        question = example["question"]
        if question in [output["question"] for output in outputs]:
            continue

        answer, relevant_docs = answer_with_rag(
            question, llm, knowledge_index, reranker=reranker
        )
        if verbose:
            print("=======================================================")
            print(f"Question: {question}")
            print(f"Answer: {answer}")
            print(f'True answer: {example["answer"]}')
        result = {
            "question": question,
            "true_answer": example["answer"],
            "source_doc": example["source_doc"],
            "generated_answer": answer.content, # Fixed: Accessing the content attribute
            "retrieved_docs": [doc for doc in relevant_docs],
        }
        if test_settings:
            result["test_settings"] = test_settings
        outputs.append(result)

        with open(output_file, "w") as f:
            json.dump(outputs, f)

In [ ]:
EVALUATION_PROMPT = """###Task Description:
An instruction (might include an Input inside it), a response to evaluate, a reference answer that gets a score of 5, and a score rubric representing a evaluation criteria are given.
1. Write a detailed feedback that assess the quality of the response strictly based on the given score rubric, not evaluating in general.
2. After writing a feedback, write a score that is an integer between 1 and 5. You should refer to the score rubric.
3. The output format should look as follows: "Feedback: {{write a feedback for criteria}} [RESULT] {{an integer number between 1 and 5}}"
4. Please do not generate any other opening, closing, and explanations. Be sure to include [RESULT] in your output.

###The instruction to evaluate:
{instruction}

###Response to evaluate:
{response}

###Reference Answer (Score 5):
{reference_answer}

###Score Rubrics:
[Is the response correct, accurate, and factual based on the reference answer?]
Score 1: The response is completely incorrect, inaccurate, and/or not factual.
Score 2: The response is mostly incorrect, inaccurate, and/or not factual.
Score 3: The response is somewhat correct, accurate, and/or factual.
Score 4: The response is mostly correct, accurate, and factual.
Score 5: The response is completely correct, accurate, and factual.

###Feedback:"""

from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain_core.messages import SystemMessage


evaluation_prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a fair evaluator language model."),
        HumanMessagePromptTemplate.from_template(EVALUATION_PROMPT),
    ]
)

In [ ]:
from langchain_openai import ChatOpenAI
from google.colab import userdata

eval_chat_model = ChatOpenAI(
    model="gpt-4.1",
    temperature=0,
    openai_api_key=userdata.get('key_ptn'),
    base_url="https://llm.ptnglobalcorp.com"
)
evaluator_name = "GPT4.1"

def evaluate_answers(
    answer_path: str,
    eval_chat_model,
    evaluator_name: str,
    evaluation_prompt_template: ChatPromptTemplate,
) -> None:
    """Evaluates generated answers. Modifies the given answer file in place for better checkpointing."""
    answers = []
    if os.path.isfile(answer_path):
        with open(answer_path, "r") as f:
            answers = json.load(f)

    for experiment in tqdm(answers):
        if f"eval_score_{evaluator_name}" in experiment:
            continue

        eval_prompt = evaluation_prompt_template.format_messages(
            instruction=experiment["question"],
            response=experiment["generated_answer"],
            reference_answer=experiment["true_answer"],
        )
        eval_result = eval_chat_model.invoke(eval_prompt)

        feedback, score = [
            item.strip() for item in eval_result.content.split("[RESULT]")
        ]
        experiment[f"eval_score_{evaluator_name}"] = score
        experiment[f"eval_feedback_{evaluator_name}"] = feedback

        with open(answer_path, "w") as f:
            json.dump(answers, f)

In [ ]:
if not os.path.exists("./output"):
    os.mkdir("./output")

READER_MODEL_NAME = "zephyr-7b-beta"

for chunk_size in [200]:  # Add other chunk sizes (in tokens) as needed
    for embeddings in ["thenlper/gte-small"]:  # Add other embeddings as needed
        for rerank in [True, False]:
            settings_name = f"chunk:{chunk_size}_embeddings:{embeddings.replace('/', '~')}_rerank:{rerank}_reader-model:{READER_MODEL_NAME}"
            output_file_name = f"./output/rag_{settings_name}.json"

            print(f"Running evaluation for {settings_name}:")

            print("Loading knowledge base embeddings...")
            knowledge_index = load_embeddings(
                RAW_KNOWLEDGE_BASE,
                chunk_size=chunk_size,
                embedding_model_name=embeddings,
            )

            print("Running RAG...")
            reranker = (
                RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")
                if rerank
                else None
            )

            run_rag_tests(
                eval_dataset=eval_dataset,
                llm=READER_LLM,
                knowledge_index=knowledge_index,
                output_file=output_file_name,
                reranker=reranker,
                verbose=False,
                test_settings=settings_name,
            )

In [ ]:
# Split into 2 sessions: run_rag_tests and evaluate_anwsers in order to fix bug

for chunk_size in [200]:  # Add other chunk sizes (in tokens) as needed
    for embeddings in ["thenlper/gte-small"]:  # Add other embeddings as needed
        for rerank in [True, False]:
            settings_name = f"chunk:{chunk_size}_embeddings:{embeddings.replace('/', '~')}_rerank:{rerank}_reader-model:{READER_MODEL_NAME}"
            output_file_name = f"./output/rag_{settings_name}.json"

            print("Running evaluation (GPT4.1 Judge)...")
            evaluate_answers(
                output_file_name,
                eval_chat_model,
                evaluator_name,
                evaluation_prompt_template,
            )

In [ ]:
import glob

outputs = []
for file in glob.glob("./output/*.json"):
    output = pd.DataFrame(json.load(open(file, "r")))
    output["settings"] = file
    outputs.append(output)
result = pd.concat(outputs)

In [ ]:
result["eval_score_GPT4.1"] = result["eval_score_GPT4.1"].apply(
    lambda x: int(x) if isinstance(x, str) else 1
)
result["eval_score_GPT4.1"] = (result["eval_score_GPT4.1"] - 1) / 4

In [ ]:
average_scores = result.groupby("settings")["eval_score_GPT4.1"].mean()
average_scores.sort_values()

In [ ]:
import plotly.express as px

scores = datasets.load_dataset("m-ric/rag_scores_cookbook", split="train")
scores = pd.Series(scores["score"], index=scores["settings"])

In [ ]:
fig = px.bar(
    scores,
    color=scores,
    labels={
        "value": "Accuracy",
        "settings": "Configuration",
    },
    color_continuous_scale="bluered",
)
fig.update_layout(
    width=1000,
    height=600,
    barmode="group",
    yaxis_range=[0, 100],
    title="<b>Accuracy of different RAG configurations</b>",
    xaxis_title="RAG settings",
    font=dict(size=15),
)
fig.layout.yaxis.ticksuffix = "%"
fig.update_coloraxes(showscale=False)
fig.update_traces(texttemplate="%{y:.1f}", textposition="outside")
fig.show()